In [43]:
import random
import nltk
import pickle
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode

documents = [(list(movie_reviews.words(fileid)), category)
           for category in movie_reviews.categories()
           for fileid in movie_reviews.fileids(category)]


random.shuffle(document)   
all_words = []
for w in movie_reviews.words():
    all_words.append(w.lower())
    
all_words = nltk.FreqDist(all_words)
#print(all_words.most_common(15))
#print(all_words["stupid"])


word_features = list(all_words.keys())[:3000]
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features :
        features[w] = (w in words)
        
    return features

#print(find_features(movie_reviews.words('neg/cv000_29416.txt')))

featuresets = [(find_features(rev), category) for (rev, category) in document]

training_set = featuresets[:1900]
test_set = featuresets[1900:]


classifier = nltk.NaiveBayesClassifier.train(training_set)
print("original naive bayes accuracy :",(nltk.classify.accuracy(classifier,test_set))*100)
classifier.show_most_informative_features(10)



original naive bayes accuracy : 81.0
Most Informative Features
                   sucks = True              neg : pos    =     10.0 : 1.0
                  annual = True              pos : neg    =      9.8 : 1.0
                  turkey = True              neg : pos    =      8.6 : 1.0
                 frances = True              pos : neg    =      8.5 : 1.0
           unimaginative = True              neg : pos    =      8.2 : 1.0
                 idiotic = True              neg : pos    =      7.3 : 1.0
             silverstone = True              neg : pos    =      6.9 : 1.0
                  kombat = True              neg : pos    =      6.9 : 1.0
               atrocious = True              neg : pos    =      6.9 : 1.0
                  regard = True              pos : neg    =      6.7 : 1.0


In [ ]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy :",(nltk.classify.accuracy(MNB_classifier,test_set))*100)

#GaussianNB, BernoulliNB


BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy :",(nltk.classify.accuracy(BernoulliNB_classifier,test_set))*100)

#LogisticRegression, SGDClassifier
#SVC, LinearSVC, NuSVC
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy :",(nltk.classify.accuracy(LogisticRegression_classifier,test_set))*100)


SGDClassifier = SklearnClassifier(SGDClassifier())
SGDClassifier.train(training_set)
print("SGDClassifier accuracy :",(nltk.classify.accuracy(SGDClassifier,test_set))*100)



SVC_classifier = SklearnClassifier(SVC())
SVC_classifier.train(training_set)
print("SVC_classifier accuracy :",(nltk.classify.accuracy(SVC_classifier,test_set))*100)



LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy :",(nltk.classify.accuracy(LinearSVC_classifier,test_set))*100)




NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier accuracy :",(nltk.classify.accuracy(NuSVC_classifier,test_set))*100)

class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers
        
    def classify(self,features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
            
    def confidence(self,features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
            
        choice_votes = votes.count(mode(votes))
        conf = choice_votes/len(votes)
        return conf
        



voted_classifier = VoteClassifier(classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier,
                                  SGDClassifier,
                                  SVC_classifier,
                                  LinearSVC_classifier,
                                  NuSVC_classifier)
                                 


print("voted classifier accuracy :",(nltk.classify.accuracy(voted_classifier,test_set))*100)

print("Classification :", voted_classifier.classify(test_set[1][0]), "confidence %:",
      voted_classifier.confidence(test_set[1][0]))



MNB_classifier accuracy : 83.0


In [ ]:
import pickle
save_classifier = open("NLP_naivebayes.pickle","wb")
pickle.dump(classifier,save_classifier)
save_classifier.close()